In [1]:
import pypyodbc as odbc

# Correctly define variables
DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = r'Pratiksharaut\SQLEXPRESS'  # raw string avoids \S escape issue
DATABASE_NAME = 'inventory'

# Properly format the connection string
connection_string = (
    f"DRIVER={{{DRIVER_NAME}}};"
    f"SERVER={SERVER_NAME};"
    f"DATABASE={DATABASE_NAME};"
    f"Trusted_Connection=yes;"
)

# Connect

conn = odbc.connect(connection_string)
print(conn)

In [2]:
import pyodbc
from tkinter import messagebox

# Create the connection outside so you don't open/close repeatedly
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=PRATIKSHARAUT\\SQLEXPRESS;'
    'DATABASE=inventory;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

In [ ]:
import tkinter as tk
from tkinter import ttk, Frame, Label, Button, Entry, Checkbutton, IntVar, LabelFrame, messagebox, StringVar
from tkinter.constants import X, BOTH, LEFT, GROOVE, RAISED, W
from tkcalendar import DateEntry
current_row = 0
list_price_entries = []

def Bill_entry():
    root = tk.Tk()
    root.geometry("1350x750+0+0")
    root.title("Billing Management System")

    top_frame = Frame(root, bg="#0f4d7d", bd=7, relief=GROOVE)
    top_frame.pack(fill=X)

    Button(top_frame, text="← Back", font=("Arial", 12, "bold"), bg="white", fg="#0f4d7d",
           bd=3, relief=RAISED, command=root.destroy).pack(side=LEFT, padx=10, pady=5)

    Label(top_frame, text="Billing Entry", font=('Times new roman', 30, "bold"),
          bg="#0f4d7d", fg="white").pack(side=LEFT, padx=20)

    Billing_frame = Frame(root, bg="white", bd=5, relief=GROOVE)
    Billing_frame.pack(fill=BOTH, expand=True)

    F1 = LabelFrame(Billing_frame, text="Customer Details", bd=5, relief=GROOVE,
                    font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")
    F1.place(x=0, y=0, relwidth=1, height=120)

    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    billno_var = tk.StringVar()
    address_var = tk.StringVar()

   
    Label(F1, text="Customer Name:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=0, column=0, padx=20, pady=5)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=name_var).grid(row=0, column=1, padx=10, pady=5)

    Label(F1, text="Phone No.:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=0, column=2, padx=20, pady=5)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=phone_var).grid(row=0, column=3, padx=10, pady=5)

    Label(F1, text="Bill Number:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=0, column=4, padx=20, pady=5)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=billno_var).grid(row=0, column=5, padx=10, pady=5)

    # Your existing fetch_customer_details with minor adjustment
    def fetch_customer_details():
        billno = billno_var.get().strip()
        customer_name = name_var.get().strip()

        if not billno and not customer_name:
            messagebox.showwarning("Input Error", "Please enter either Bill Number or Customer Name")
            return

        try:
            found = False

            if billno:
                cursor.execute("SELECT CustomerName, PhoneNo, Address FROM Customers WHERE BillNumber = ?", (billno,))
                row = cursor.fetchone()
                if row:
                    name_var.set(row[0])
                    phone_var.set(row[1])
                    address_var.set(row[2])
                    found = True

            if not found and customer_name:
                cursor.execute("SELECT BillNumber, PhoneNo, Address FROM Customers WHERE LOWER(CustomerName) LIKE ?", (f"%{customer_name.lower()}%",))
                row = cursor.fetchone()
                if row:
                    billno_var.set(row[0])
                    phone_var.set(row[1])
                    address_var.set(row[2])
                    name_var.set(customer_name)
                    found = True

            if not found:
                messagebox.showerror("Not Found", "No customer found with the given details.")
                billno_var.set("")
                name_var.set("")
                phone_var.set("")
                address_var.set("")

        except Exception as e:
            messagebox.showerror("Database Error", f"Error fetching data: {e}")

    Button(F1, text="Search", font=("Arial", 12, "bold"), bg="white", fg="#0f4d7d",
           command=fetch_customer_details).grid(row=0, column=6, padx=10, pady=5)

    # --- Live Search Popup for Customer Name ---
    def show_customer_suggestions(event):
        typed = name_var.get().strip()
        if not typed:
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()
            return

        try:
            # Search customers starting with typed letters (case-insensitive)
            cursor.execute("SELECT CustomerName FROM Customers WHERE LOWER(CustomerName) LIKE ? ORDER BY CustomerName ASC", (typed.lower() + '%',))
            results = cursor.fetchall()
            names = [row[0] for row in results]

            if not names:
                if hasattr(root, 'cust_popup'):
                    root.cust_popup.destroy()
                return

            # Destroy old popup if exists
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()

            popup = tk.Toplevel(root)
            root.cust_popup = popup
            popup.wm_overrideredirect(True)  # No window decorations
            x = name_entry.winfo_rootx()
            y = name_entry.winfo_rooty() + name_entry.winfo_height()
            popup.geometry(f"+{x}+{y}")

            listbox = tk.Listbox(popup, font=("arial", 12), width=30, height=min(len(names), 10))
            listbox.pack()

            for name in names:
                listbox.insert(tk.END, name)

            def on_select(event):
                if listbox.curselection():
                    selected_name = listbox.get(listbox.curselection())
                    name_var.set(selected_name)

                    # Fetch details for selected customer name
                    try:
                        cursor.execute("SELECT BillNumber, PhoneNo, Address FROM Customers WHERE CustomerName = ?", (selected_name,))
                        row = cursor.fetchone()
                        if row:
                            billno_var.set(row[0])
                            phone_var.set(row[1])
                            address_var.set(row[2])
                        else:
                            billno_var.set("")
                            phone_var.set("")
                            address_var.set("")
                    except Exception as e:
                        messagebox.showerror("Database Error", f"Error fetching data: {e}")

                    popup.destroy()
                    name_entry.focus_set()

            listbox.bind("<<ListboxSelect>>", on_select)

            # Close popup when focus lost
            popup.bind("<FocusOut>", lambda e: popup.destroy())

        except Exception as e:
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()

    name_var = tk.StringVar()
    # Define name_entry BEFORE you bind events to it
    name_entry = Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=name_var)
    name_entry.grid(row=0, column=1, padx=10, pady=5)
    
    # Then bind the event after name_entry is defined
    name_entry.bind("<KeyRelease>", show_customer_suggestions)



    Button(F1, text="Search", font=("Arial", 12, "bold"), bg="white", fg="#0f4d7d",
           command=fetch_customer_details).grid(row=0, column=6, padx=10, pady=5)

    Label(F1, text="Shipping Address:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=0, padx=5, pady=5, sticky=W)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=address_var)\
        .grid(row=1, column=1, padx=5, pady=5)

    Label(F1, text="Bill Date:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=2, padx=5, pady=5)
    DateEntry(F1, font=("arial", 15), bd=3, relief=GROOVE, background='darkblue',
              foreground='white', date_pattern='dd/mm/yyyy')\
        .grid(row=1, column=3, padx=5, pady=5, sticky=W)

    Label(F1, text="Delivery Date:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=4, padx=5, pady=5)
    DateEntry(F1, font=("arial", 15), bd=3, relief=GROOVE, background='darkblue',
              foreground='white', date_pattern='dd/mm/yyyy')\
        .grid(row=1, column=5, padx=5, pady=5, sticky=W)
     
    def check_customer_details_filled():
            name = name_var.get().strip()
            contact = contact_var.get().strip()
            address = address_var.get().strip()
        
            missing_fields = []
            if not name:
                missing_fields.append("Name")
            if not contact:
                missing_fields.append("Contact")
            if not address:
                missing_fields.append("Address")
        
            if missing_fields:
                messagebox.showerror(
                    "Missing Customer Details",
                    f"Please fill in the following fields before proceeding: {', '.join(missing_fields)}"
                )
                return False
            return True
    Label(F1, text="Cash Disc:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=6, padx=5)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, width=4).grid(row=1, column=7, padx=5, sticky=W)

    def on_discount_enter(event, row):
        try:
            discount_text = row[7].get().strip()
            qty = float(row[4].get())
            price = float(row[6].get())
    
            if discount_text.endswith('%'):
                discount_val = float(discount_text[:-1])
                unit_price = price - (price * discount_val / 100)
            else:
                discount_val = float(discount_text)
                unit_price = price - discount_val
    
            # Update DISC UNIT PRICE (column 8)
            row[8].config(state="normal")
            row[8].delete(0, tk.END)
            row[8].insert(0, f"{unit_price:.2f}")
            row[8].config(state="readonly")
    
            # Update DISC GROSS PRICE (column 9)
            gross_price = unit_price * qty
            row[9].config(state="normal")
            row[9].delete(0, tk.END)
            row[9].insert(0, f"{gross_price:.2f}")
            row[9].config(state="readonly")
    

    
        except Exception as e:
            print(f"Error in discount entry: {e}")
     # ---------- Row Logic ----------
    entry_rows = []
    current_row = 1
    
    def update_lp_total():
        total = 0.0
        for row in entry_rows:
            val = row[10].get()
            try:
                if val:
                    total += float(val)
            except ValueError:
                pass
        lp_total_var.set(f"{total:.2f}")
    
    def lp_price(event, row):
        try:
            qty = float(row[4].get())
            price = float(row[6].get())
    
            if qty < 0 or price < 0:
                raise ValueError("Quantity and Price must be non-negative.")
    
            total_price = qty * price
            row[10].config(state="normal")
            row[10].delete(0, tk.END)
            row[10].insert(0, f"{total_price:.2f}")
            row[10].config(state="readonly")
    
        except ValueError as ve:
            row[10].config(state="normal")
            row[10].delete(0, tk.END)
            row[10].insert(0, "")
            row[10].config(state="readonly")
         
        update_lp_total()

    def apply_cash_discount():
        try:
            discount_val = float(cash_disc_var.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Enter a valid cash discount.")
            return
    
        is_percent = cash_chk_var.get() == 1
        total_gross_price = 0
    
        # Calculate total gross price (sum of all row gross prices)
        for row in entry_rows:
            try:
                gross = float(row[9].get())  # Assuming row[9] holds the gross price of that row
                total_gross_price += gross
            except Exception:
                continue
    
        # Apply cash discount on overall total gross price
        if is_percent:
            discounted_total = total_gross_price * (1 - discount_val / 100)
        else:
            discounted_total = total_gross_price - discount_val
            if discounted_total < 0:
                discounted_total = 0  # Avoid negative total
    
        # Update net discounted total price in F3 Entry
        net_disc_price_var.set(f"₹ {discounted_total:.2f}")

        """
    def update_total_price():
            total = 0
            for row in entry_rows:
                try:
                    gross = float(row[9].get())
                    total += gross
                except:
                    continue
            lp_total_var.set(f"₹ {total:.2f}")
        
        
        """      
        
    
        


    Button(F1, text="Appl ✅", font=("Arial", 10, "bold"), bg="white", fg="#0f4d7d",
       width=10, anchor="center", command=apply_cash_discount).grid(row=1, column=8, padx=5)


    cash_disc_var = StringVar(value="0")
    cash_chk_var = IntVar()
    Entry(F1, textvariable=cash_disc_var, font=("arial", 15), bd=3, relief=GROOVE, width=4).grid(row=1, column=7, padx=5, sticky=W)
    
    Checkbutton(F1, text="%", variable=cash_chk_var, bg="#0f4d7d", font=("Arial", 18, "bold"),
                activebackground="white", fg="black", selectcolor="white").grid(row=1, column=8, sticky=W)

    F2 = LabelFrame(Billing_frame, bd=5, relief=GROOVE, font=('times new roman', 15, "bold"),
                    bg="white", fg="gold")
    F2.place(x=0, y=110, relwidth=1, height=450)
    
    canvas = tk.Canvas(F2, bg="white")
    canvas.pack(side=tk.TOP, fill=BOTH, expand=True)
    
    h_scrollbar = tk.Scrollbar(F2, orient=tk.HORIZONTAL, command=canvas.xview)
    h_scrollbar.pack(side=tk.BOTTOM, fill=X)
    
    canvas.configure(xscrollcommand=h_scrollbar.set)
    
    scrollable_frame = Frame(canvas, bg="white")
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    
    def on_configure(event):
        canvas.configure(scrollregion=canvas.bbox("all"))
    
    scrollable_frame.bind("<Configure>", on_configure)
    
    headers = [
    "DESCRIPTION", "RATINGS", "CAT NO.", "MAKE", "QUANTITY",
    "NET QTY", "LIST PRICE", "DISCOUNT", "DISC UNIT PRICE", "DISC GROSS PRICE","LP GRASS.PRICE"
    ]

    for col, text in enumerate(headers):
        lbl = Label(scrollable_frame, text=text, font=('times new roman', 13),
                    bg="#A1AEB1", fg="black", width=20)
        lbl.grid(row=0, column=col, padx=1, pady=0, sticky='w')
    
    # The big Entry spanning all columns
    B1 = tk.Entry(scrollable_frame, font=('times new roman', 12, "bold"),
                  bg="#DBE2E9", fg="black", width=100)
    B1.grid(row=1, column=0, columnspan=len(headers), sticky='we', padx=1, pady=0)

    # Configure grid column 0 to expand, others not
    scrollable_frame.grid_columnconfigure(0, weight=1)
    for col in range(1, len(headers)):
        scrollable_frame.grid_columnconfigure(col, weight=0)
  
    # Entry widgets in row 2
    entries = []
    for col in range(len(headers)):
        entry = tk.Entry(scrollable_frame, font=("arial", 12), bd=3, relief=tk.GROOVE, width=20)
        entry.grid(row=2, column=col, padx=1, pady=1)
        entries.append(entry)
    
    entry_rows = []
    current_row = 2  # After header row and B1

    def on_qty_enter(event, row_entries):
        # Only add a new row if this is the last one
        if row_entries == entry_rows[-1]:
            add_entry_row()

    def add_section_heading_row():
        nonlocal current_row
        
        # Add a big Entry spanning all columns like B1
        section_entry = tk.Entry(scrollable_frame, font=('times new roman', 12, "bold"),
                                 bg="#DBE2E9", fg="gold", width=100)
        section_entry.grid(row=current_row, column=0, columnspan=len(headers), sticky='we', padx=1, pady=0)
        
        # Bind Enter key to add normal entry row below this heading
        def on_section_enter(event):
            if section_entry.get().strip() != "":
                add_entry_row()
                section_entry.config(state='disabled')  # Disable editing once section heading is entered
                section_entry.unbind("<Return>")  # Remove binding after use
            return "break"  # Prevent default behavior
        
        section_entry.bind("<Return>", on_section_enter)
        
        current_row += 1

    def on_qty_enter(event, row_entries):
        # Only add a new row if this is the last one
        if row_entries == entry_rows[-1]:
            qty_val = row_entries[4].get().strip()
            if qty_val == "":
                # Add section heading row instead of normal entry row
                add_section_heading_row()
            else:
                add_entry_row()
    
    def add_entry_row():
        nonlocal current_row
    
        row_entries = []
        readonly_columns = (8, 9, 10)  # DISC UNIT, DISC GROSS, LP GROSS.PRICE
    
        for col in range(len(headers)):
            state = "readonly" if col in readonly_columns else "normal"
            e = tk.Entry(scrollable_frame, font=("arial", 12), bd=3,
                         relief=tk.GROOVE, width=20, state=state, fg="black")
            e.grid(row=current_row, column=col, padx=1, pady=1)
            row_entries.append(e)
    
        # Bind popup to DESCRIPTION
        row_entries[0].bind("<KeyRelease>", lambda event, r=row_entries: show_popup(event, r))
    
        # QUANTITY (index 4) — bind both Enter and FocusOut
        row_entries[4].bind("<Return>", lambda event, r=row_entries: on_qty_enter(event, r))
        row_entries[4].bind("<FocusOut>", lambda event, r=row_entries: lp_price(event, r))
    
        # LIST PRICE (index 6) — bind FocusOut for LP calculation
        row_entries[6].bind("<FocusOut>", lambda event, r=row_entries: lp_price(event, r))
    
        # DISCOUNT (index 7) — bind Enter key
        row_entries[7].config(state="normal")
        row_entries[7].bind("<Return>", lambda event, r=row_entries: on_discount_enter(event, r))
    
        entry_rows.append(row_entries)
        current_row += 1
    

    inventory = [  # Add this dummy inventory list
    {"description": "MCCB 100A", "make": "ABB", "qty": 5, "ratings": "100A", "catno": "ABB100A", "listprice":26710, "net_qty":1},
    {"description": "MCB 32A", "make": "Siemens", "qty": 10, "ratings": "32A", "catno": "SIE32A", "listprice":400, "net_qty":1},
    {"description": "Contactors 9A", "make": "Schneider", "qty": 12, "ratings": "9A", "catno": "SCH9A", "listprice":750, "net_qty":1},
    {"description": "TPN DB 4WAY", "make": "Legrand", "qty": 3, "ratings": "4W", "catno": "LEG4W", "listprice":50, "net_qty":1},
    {"description": "Cable 10sqmm", "make": "Polycab", "qty": 20, "ratings": "10sqmm", "catno": "POL10SQ", "listprice":200, "net_qty":1},
    {"description": "ELCB 40A", "make": "Anchor", "qty": 60, "ratings": "40A","catno": "ELCB40", "listprice":680, "net_qty":1},
    ]

    def show_popup(event, entry_list):
        typed = entry_list[0].get().lower()
        if not typed:
            return
    
        matches = [item for item in inventory if item["description"].lower().startswith(typed)]
        if not matches:
            return
    
        popup = tk.Toplevel()
        popup.wm_title("Select Item")
        popup.geometry("550x200+%d+%d" % (entry_list[0].winfo_rootx(), entry_list[0].winfo_rooty() + 30))
        popup.grab_set()
    
        tree = ttk.Treeview(popup, columns=("description", "make", "qty", "ratings", "catno", "listprice","net_qty"), show="headings")
        tree.heading("description", text="DESCRIPTION")
        tree.heading("make", text="MAKE")
        tree.heading("qty", text="QTY")
        tree.heading("ratings", text="RATINGS")
        tree.heading("catno", text="CAT NO.")
        tree.heading("listprice",text="LIST PRICE")
        tree.heading("net_qty",text="NET_QUANTITY")
    
        for item in matches:
            tree.insert("", "end", values=(
                item["description"], item["make"], item["qty"], item["ratings"], item["catno"], item["listprice"], item["net_qty"]
            ))
    
        tree.pack(fill="both", expand=True)
    
        def on_select(event):
            selected = tree.focus()
            values = tree.item(selected, "values")
            if values:
                entry_list[0].delete(0, tk.END)
                entry_list[0].insert(0, values[0])  # DESCRIPTION
    
                entry_list[1].delete(0, tk.END)
                entry_list[1].insert(0, values[3])  # RATINGS
    
                entry_list[2].delete(0, tk.END)
                entry_list[2].insert(0, values[4])  # CAT NO.
    
                entry_list[3].delete(0, tk.END)
                entry_list[3].insert(0, values[1])  # MAKE
    
                entry_list[4].delete(0, tk.END)
                entry_list[4].insert(0, values[2])  # QTY

                entry_list[5].delete(0, tk.END)
                entry_list[5].insert(0, values[6]) #listprice

                entry_list[6].delete(0, tk.END)
                entry_list[6].insert(0, values[5])

                
    
                popup.destroy()
    
        tree.bind("<Double-1>", on_select)
    
    # Make sure this is after `entries` is created and `show_popup` is defined
    entries[0].bind("<KeyRelease>", lambda event: show_popup(event, entries))
    
    F3 = LabelFrame(Billing_frame, text="Bill Menu", bd=5, relief=GROOVE,
                    font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")
    F3.place(x=0, y=560, relwidth=1, height=160)
    
    def open_enclosure_popup(total_var):
        popup = tk.Toplevel()
        popup.title("Enclosure Fabrication")
        popup.geometry("1100x700")
        popup.configure(bg="white")
    
        # ---------- First Table Headers ----------
        headers = [" ", "MM", "Length", "Width", "", "Kg", "", "Kg per Sqmm"]
    
        enclosure_data = [
            {" ": "18 SWG", "MM": 1.20, "Length": 2500, "Width": 1250, "Kg": 30, "": 29.4375, "Kg per Sqmm": 0.0000096},
            {" ": "16 SWG", "MM": 1.50, "Length": 2500, "Width": 1250, "Kg": 40, "": 36.796875, "Kg per Sqmm": 0.0000128},
            {" ": "14 SWG", "MM": 2.00, "Length": 2500, "Width": 1250, "Kg": 50, "": 49.0625, "Kg per Sqmm": 0.000016},
            {" ": "13 SWG", "MM": 2.30, "Length": 2500, "Width": 1250, "Kg": 60, "": 56.421875, "Kg per Sqmm": 0.0000192},
            {" ": "12 SWG", "MM": 2.64, "Length": 2500, "Width": 1250, "Kg": 70, "": 64.7625, "Kg per Sqmm": 0.0000224},
            {" ": "11 SWG", "MM": 3.00, "Length": 2500, "Width": 1250, "Kg": 80, "": 73.59375, "Kg per Sqmm": 0.0000256},
            {" ": "10 SWG", "MM": 3.25, "Length": 2500, "Width": 1250, "Kg": 90, "": 79.7265625, "Kg per Sqmm": 0.0000288},
        ]
    
        # Header labels for first table
        for col, text in enumerate(headers):
            tk.Label(popup, text=text, font=('times new roman', 12, "bold"),
                     bg="#A1AEB1", fg="black", width=20).grid(row=0, column=col, padx=1, pady=2)
    
        # Data entries for first table
        entry_matrix1 = []
        for row_index, data in enumerate(enclosure_data):
            row_entries = []
            for col_index, header in enumerate(headers):
                ent = tk.Entry(popup, font=('times new roman', 12), width=20)
                ent.grid(row=row_index + 1, column=col_index, padx=1, pady=1)
    
                if header in data:
                    ent.insert(0, data[header])
                if row_index in [0, 2, 4, 6]:
                    ent.config(bg="#FF7F7F")  # Highlight specific rows
                row_entries.append(ent)
            entry_matrix1.append(row_entries)
    
        # ---------- Second Table Headers ----------
        headers1 = ["", "L", "W", "H"]
        second_table_row_start = len(enclosure_data) + 4  # Leave one empty row

        Enclosure_data1 = [
            {"":28+1+1,"L":2400,"W":300,"H":1600},
        ]
            

        
    
        for col, text in enumerate(headers1):
            tk.Label(popup, text=text, font=('times new roman', 12, "bold"),
                     bg="#A1AEB1", fg="black", width=20).grid(row=second_table_row_start, column=col, padx=1, pady=2)
    
        # Entry fields for manual dimensions
        entry_matrix2 = []
        for row in range(5):  # 5 rows
            row_entries = []
            for col in range(len(headers1)):
                ent = tk.Entry(popup, font=('times new roman', 12), bg="white", width=20)
                ent.grid(row=second_table_row_start + row + 1, column=col, padx=1, pady=1)
                row_entries.append(ent)
            entry_matrix2.append(row_entries)
    
        # ---------- Total Calculation ----------
        def enclosure_calculate_total():
            total_volume = 0
            for row in entry_matrix2:
                try:
                    l = float(row[0].get())
                    w = float(row[1].get())
                    h = float(row[2].get())
                    total_volume += l * w * h
                except ValueError:
                    continue
            total_var.set(f"{total_volume:.2f}")
    
        # ---------- Buttons ----------
        btn_row = second_table_row_start + 6 + 1  # After 5 manual rows + header
    
        tk.Button(popup, text="Total", command=enclosure_calculate_total,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white")\
                  .grid(row=btn_row, column=1, pady=10, padx=10, sticky="w")
    
        tk.Button(popup, text="Close", command=popup.destroy,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white")\
                  .grid(row=btn_row, column=0, pady=10, padx=10, sticky="e")

    enclosure_total_var = tk.StringVar(value="0.00")


    Button(F3, text="ENCLOSURE FABRICATION:", font=('times new roman', 12, "bold"),
           bg="#0f4d7d", fg="gold", command=lambda: open_enclosure_popup(enclosure_total_var))\
           .grid(row=0, column=0, padx=20, pady=5)

    Label(F3, textvariable=enclosure_total_var, font=("arial", 12), bd=3,
          relief=GROOVE, width=20).grid(row=0, column=1, padx=10, pady=5)
    
    wiring_total_var = tk.StringVar(value="₹ 0.00")
 
    def open_wiring_popup(wiring_total_var):
        popup = tk.Toplevel()
        popup.title("Wiring + Accessories")
        popup.geometry("600x400")
        popup.configure(bg="white")
    
        headers = ["SIZE", "RATE", "QTY", "TOTAL"]
    
        predefined_items = [
            (35, 350),
            (25, 250),
            (16, 170),
            (10, 110),
            (6, 60),
            (4, 40),
            (2.5, 30),
            (1, 15)
        ]
    
        canvas = tk.Canvas(popup, bg="white")
        frame = tk.Frame(canvas, bg="white")
        scrollbar = tk.Scrollbar(popup, orient="vertical", command=canvas.yview)
        canvas.configure(yscrollcommand=scrollbar.set)
    
        scrollbar.pack(side="right", fill="y")
        canvas.pack(side="left", fill="both", expand=True)
        canvas.create_window((0, 0), window=frame, anchor='nw')
    
        def on_configure(event):
            canvas.configure(scrollregion=canvas.bbox("all"))
    
        frame.bind("<Configure>", on_configure)
    
        for col, text in enumerate(headers):
            lbl = tk.Label(frame, text=text, font=('times new roman', 12, "bold"),
                           bg="#A1AEB1", fg="black", width=20)
            lbl.grid(row=0, column=col, padx=1, pady=2)
    
        entry_matrix = []
    
        def wiring_calculate_total():
            total = 0
            for row in entry_matrix:
                try:
                    qty = float(row[2].get())
                    rate = float(row[1].get())
                    row_total = qty * rate
                    row[3].config(state="normal")
                    row[3].delete(0, tk.END)
                    row[3].insert(0, f"{row_total:.2f}")
                    row[3].config(state="readonly")
                    total += row_total
                except:
                    row[3].config(state="normal")
                    row[3].delete(0, tk.END)
                    row[3].insert(0, "")
                    row[3].config(state="readonly")
                    continue
            wiring_total_var.set(f"₹ {total:.2f}")
    
        def wiring_add_row(size="", rate=""):
            row_idx = len(entry_matrix) + 1
            entries = []
    
            # SIZE Entry
            e1 = tk.Entry(frame, width=20)
            e1.insert(0, size)
            e1.grid(row=row_idx, column=0, padx=1, pady=2)
            entries.append(e1)
    
            # RATE Entry
            e2 = tk.Entry(frame, width=20)
            e2.insert(0, rate)
            e2.grid(row=row_idx, column=1, padx=1, pady=2)
            entries.append(e2)
    
            # QTY Entry
            e3 = tk.Entry(frame, width=20)
            e3.grid(row=row_idx, column=2, padx=1, pady=2)
            entries.append(e3)
    
            # TOTAL (readonly) 
            e4 = tk.Entry(frame, width=20, state="readonly")
            e4.grid(row=row_idx, column=3, padx=1, pady=2)
            entries.append(e4)
    
            # Bind Enter on QTY to calculate total and auto-add row
            def on_enter(event):
                wiring_calculate_total()
                if entries == entry_matrix[-1]:
                    wiring_add_row()  # Add new empty row
    
            e3.bind("<Return>", on_enter)
    
            entry_matrix.append(entries)
    
      
        for size, rate in predefined_items:
            wiring_add_row(size, rate)
    
        # Buttons
        btn_frame = tk.Frame(popup, bg="white")
        btn_frame.pack(pady=10)
    
        tk.Button(btn_frame, text="Total", command=wiring_calculate_total,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white", width=12)\
            .grid(row=0, column=0, padx=10)
    
        tk.Button(btn_frame, text="Close", command=popup.destroy,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white", width=12)\
            .grid(row=0, column=1, padx=10)

   
    # Now create the button that opens the popup
    Button(F3, text="WIRING+ACCESSORIES:", font=('times new roman', 12, "bold"),
           bg="#0f4d7d", fg="gold", command=lambda: open_wiring_popup(wiring_total_var))\
           .grid(row=1, column=0, padx=20)
    
    # And the label that shows the total
    Label(F3, textvariable=wiring_total_var, font=("arial", 12), bd=3,
          relief=GROOVE, width=20).grid(row=1, column=1, padx=10)
    

    Label(F3, text="LBOUR:", font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=2, column=0, padx=20)
    Entry(F3, font=("arial", 12), bd=3, relief=GROOVE).grid(row=2, column=1, padx=10)

    Label(F3, text="TRANSPORTATION:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=0, column=2)
    Entry(F3, font=("arial", 15), bd=3, relief=GROOVE).grid(row=0, column=3, padx=10)

    Label(F3, text="BUS:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=2)
    Entry(F3, font=("arial", 15), bd=3, relief=GROOVE).grid(row=1, column=3, padx=10)

    Button(F3, text="GENERATE BILL", font=('arial', 12, 'bold'), bg="white", fg="black", pady=5, padx=5)\
        .grid(row=2, column=6, pady=5, padx=5)

    Label(F3, font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="white")\
            .grid(row=2, column=8, padx=20, pady=5)

    lp_total_var = tk.StringVar(value="0.00")
    Label(F3, text="LP. GROSS PRICE", font=("arial", 12, "bold"), bg="#0f4d7d", fg="gold").grid(row=0, column=7, padx=10, pady=5, sticky="w")
    Entry(F3, textvariable=lp_total_var, font=("arial", 12), bd=3, state="readonly").grid(row=0, column=8, padx=10, pady=5)

    net_disc_price_var = StringVar(value="₹ 0.00")
    Label(F3, text="NET DISC. PRICE:", font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="gold")\
    .grid(row=1, column=7, padx=20, pady=5)
    Entry(F3, font=("arial", 12), bd=3, relief=GROOVE, textvariable=net_disc_price_var, state="readonly")\
    .grid(row=1, column=8, padx=10, pady=5)
    
   
    add_entry_row()
    root.mainloop()

Bill_entry()